In [3]:
import pandas as pd
from openai import OpenAI

# Set your OpenAI API key
client = OpenAI(api_key='sk-...')

def load_conversation(csv_file):
    df = pd.read_csv(csv_file, header=None)
    return df

def generate_feedback(conversation_file, topics=""):

    conversation = load_conversation(conversation_file)
    
    # Combine user prompts (column 0) and AI responses (column 1) for the prompt
    conversation_text = "\n".join([f"Student: {row[0]}\nAI: {row[1]}" for _, row in conversation.iterrows()])
    
    # Create a message for the API
    messages = [
        {"role": "system", "content": f"You are an AI grader to evaluate student asslignments for a class called Data Science Applications in Business. In this assignment, students are asked to chat with AI chatbot to learn about relevant topics. Here are the topics they are expected to learn for this assignment: {topics}"},
        {"role": "user", "content": f"""
        Below is the student's conversation with AI. Provide feedback/analysis on: quality of questioning, depth of understanding, and engagement with the topic. Grade the student's overall performance on a scale of 6 to 10, with 10 being the highest.Show the grade at the start of your response. 
        Here is the conversation:
        {conversation_text}
        """}
    ]
    
    # Call OpenAI API for feedback (non-streaming)
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=messages,
    )
    
    return response.choices[0].message.content


# load the file
file = 'transcript1.csv'

# Load topics for the assignment (copy from the class content page)
topics = """General Strategy
SWOT, Porter’s Five Forces, KPIs
The difference between Competitive Advantage and Operational Efficiency
International Strategy: What other considerations are needed?
What are network effects and how are they different from economies of scale?
Data Strategy
Keywords: Business Intelligence, Big Data, Automation, Embedded ML, Generative AI
What are each of these things, and how do they relate to each other?
What are the key technologies associated with each?
How do each of these concepts map to general business strategies? I.e., what strategic goals could I achieve using "Business Intelligence"?
Explore how a firm's culture impacts its practice of data science
What types of corporate culture work best to facilitate successful data science projects?
What kinds of activities do leaders need to engage in?
What are some warning signs that firm culture is holding you back from using data science successfully?
What is "Data Governance"? Use the prompt above but replace Business Strategy with Data Governance. """



feedback = generate_feedback('./Transcripts/transcript2.csv', topics = topics)
print(feedback)


Grade: 9

Feedback/Analysis:
1. **Quality of Questioning**: The student consistently demonstrated high-quality questioning skills. Each query was purposefully crafted to deepen understanding of complex strategic and data science concepts. For instance, they cleverly asked for real-life examples and succinct descriptions to relate theoretical frameworks to practical scenarios, improving the substantive depth of the conversation.

2. **Depth of Understanding**: The depth of understanding shown by the student was commendable. They were able to grasp intricate topics like Porter’s Five Forces, network effects versus economies of scale, and their applications in data strategies. The student’s ability to seek clarifications on overlapping concepts like the difference between competitive advantage and operational efficiency showed a keenness to differentiate and fully comprehend each idea.

3. **Engagement with the Topic**: The student was thoroughly engaged with the topics, demonstrated by t